In [1]:
# Load modules for data manipulation
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import call
import pandas as pd
import numpy as np
import time
import os

In [2]:
# Load modules for machine learning
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Calculate the accurancy of the model
from sklearn.metrics import accuracy_score, classification_report

# For drawing the graph
from sklearn.tree import export_graphviz

In [3]:
load_dotenv()

True

## Load the data

In [4]:
# load clients
clt = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TR35.csv', encoding='ISO-8859-1', sep='\t', low_memory=False)
# load transaction
ctr = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TR37.csv', encoding='ISO-8859-1', sep='\t', low_memory=False)
# load ecological products
eco = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'eco-products.csv', encoding='ISO-8859-1', sep='\t')

In [6]:
# Load TJ39, TJDR, TJER
tj39 = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tj39.csv', encoding='ISO-8859-1', sep=';', usecols=['COMAX', 'MSMENC', 'MTPATR', 'MTVAOP'])
tjdr = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjdr.csv', encoding='ISO-8859-1', sep=';', usecols=['COMAX', 'MTRVIM', 'MTRVFR', 'QTPAFI', 'COHAVI'])
tjer = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjer.csv', encoding='ISO-8859-1', sep=';', usecols=['COMAX', 'MTAPJE', 'MTPJE'])

## Get the best ecological product

In [5]:
# Get the list of all the eco products code
eco = [str(i).zfill(5) for i in eco['COPRO']]

# Declare the list of products for each family
# Les familles sont: CREDITS, DEPOTS MONETAIRES, PRODUITS EXTERNES COMMERCIALISES
credit = ['00568', '00943', '00942', '00546', '00547', '03992', '00940', '00941', '00548', '06458', '00509', '00565']
depot = ['07648', '07649', '07606', '00003']
comm = ['05808', '05807']

# ECO: 00003 | 242568 - LIVRET DEVELOPPEMENT DURABLE ET SOLIDAIRE
# DEPOTS MONETAIRES - EPARGNE MONETAIRE LIQUIDE - LIVRETS REGLEMENTES
# ctr[ctr['COPRO'].isin(eco)]['COPRO'].value_counts()

In [11]:
# pour le cas de la table tj39, la variable COMAX elle est de 13 caracteres
# or celle des autres tables sont de 10, donc il fallait convertir de 13 a 10
# on prend les 10 premiers caractères du COMAX de tj39, d'apres le data engineer
# de la PBS.
tj39['COMAX'] = [val[:10] for val in tj39['COMAX'].values]

In [12]:
# This table is only good fot the moral clients only
data = pd.merge(clt,tj39,on='COMAX',how='left')
data = pd.merge(data,tjer,on='COMAX',how='left')
data = pd.merge(data,tjdr,on='COMAX',how='left')

In [14]:
# Drop the duplicates
data = data.drop_duplicates(subset="COMAX")

In [37]:
# These values are based int but detected as object
# So convert them back to int
why = ['CTCOPO', 'CTMENB', 'CTSIFA', 'CTSC90', 'CTSC91', 'CTSC92', 'CTFORT','PSGPAR',
'CEBPF1', 'CEBPF2', 'CEBPF3', 'CEBPF4', 'CEBPF5', 'CEBPF6', 'CEBPF7', 'CTBP']

for i in why:
    for k in [j for j in data[i].unique() if j.strip() == '']:
         data[i].replace(k, np.nan, inplace=True)

In [29]:
# Define the list of the columns that we need
columns = ['LON', 'LAT', 'CTCOPO', 'CTMENB', 'CTSIFA', 'CTSCPI', 'CODCSP', 'COSEXE', 'QTAGCL', 'NBCART', 'QCCEL', 'QCPEL', 'QCPEP', 'QCCP19', 'QCCP21', 'QCIHAB', 'QTIAUT', 'QCITOT', 'QCASV', 'QCCTO', 'QCPEA', 'QCCRIM', 'QCCRPE', 'QCCAR1', 'QCCAR2', 'QCCAR3', 'QCCARP', 'QCCARI', 'QCBASO', 'CECARP', 'QCCOEQ', 'MTCDIM', 'MTRECD', 'MTRETT', 'QCPRDI', 'QCPRCP', 'QCPRQU', 'QCPRFI', 'QCPRTR', 'MTECEL', 'MTEPEL', 'MTECSL', 'MTEPEP', 'MTELEP', 'MTECIM', 'MTECCS', 'MTECEQ', 'MTECTR', 'MTEEML', 'MTEASV', 'MTECTO', 'MTECPA', 'MTE19D', 'MTE21D', 'MTEEMC', 'MTESOC', 'MCTOTA', 'MCTOTE', 'CTSC90', 'CTSC91', 'CTSC92', 'QCCOCY', 'QTCLIF', 'CTFORT', 'PSGPAR', 'SGMFM', 'SGMFD', 'SGMIM', 'SGMID', 'CEBPF1', 'CEBPF2', 'CEBPF3', 'CEBPF4', 'CEBPF5', 'CEBPF6', 'CEBPF7', 'CTBP', 'QCPREV', 'QCIARD', 'MTFLPR', 'MTRSFI', 'MTRSMO', 'QFPROSGR', 'MSMENC', 'MTPATR', 'MTVAOP', 'MTRVIM', 'MTRVFR', 'QTPAFI', 'MTAPJE', 'MTPJE']

In [48]:
# Fill the missing data, by the median value
data[columns] = data[columns].fillna(data[columns].median())

In [51]:
# Define the ECO variable
# First, get the ids of the products, and then assign it
ids = ctr[ctr['COPRO'].isin(eco)]['COMAX'].values
data['ECO'] = data['COMAX'].isin(ids)

## Save the data

In [58]:
# Save the data, so you can open stats file
%store data
%store columns
%store eco

Stored 'data' (DataFrame)
Stored 'columns' (list)
Stored 'eco' (list)
